In [1]:
from __future__ import annotations
from tqdm import tqdm
from dataclasses import dataclass
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re

sns.set()

with open('data.txt') as file:
    data = file.read().split('\n\n')

In [2]:
def print_board(matrix):
    for row in matrix:
        for col in row:
            print(col, end='')
        print()

def get_matrix():
    map_data = data[0].splitlines()
    height = len(map_data)
    width = max([len(m) for m in map_data])

    matrix = np.zeros((height, width), 'str')
    matrix[:, :] = ' '

    for row, line in enumerate(map_data):
        for col, point in enumerate(line):
            matrix[row, col] = point

    return matrix

In [207]:
def move_forward(matrix, pos, steps, dir):

    for step in range(steps):
        new_pos = pos + dir

        if (new_pos >= 0).all() and (new_pos < matrix.shape).all():
            point_type = matrix[new_pos[0], new_pos[1]]
        else:
            point_type = ' '

        match point_type:
            case " ":

                # Wrap map
                temp_pos = new_pos - dir
                temp_point = matrix[temp_pos[0], temp_pos[1]]

                while temp_point != " ":

                    temp_pos = temp_pos - dir

                    if (temp_pos >= 0).all() and (temp_pos < matrix.shape).all():
                        temp_point = matrix[temp_pos[0], temp_pos[1]]
                    else:
                        break

                new_pos = temp_pos + dir
                point_type = matrix[new_pos[0], new_pos[1]]

                if point_type == "#":
                    break

                pos = new_pos

            case "#":
                break
            case ".":
                pos = new_pos

    return pos


In [403]:
matrix = get_matrix()

start_col = (matrix[0] != " ").argmax()

pos = np.array([0, start_col])
dir = (0, 1)

dir_lookup = {
    (0,1): {
        'R': (1, 0),
        'L': (-1, 0)
    },
    (0,-1): {
        'R': (-1, 0),
        'L': (1, 0)
    },
    (1,0): {
        'R': (0, -1),
        'L': (0, 1)
    },
    (-1,0): {
        'R': (0, 1),
        'L': (0, -1)
    }
}

dir_score = {
    (0,1): 0,
    (1,0): 1, 
    (0,-1): 2, 
    (-1,0): 0,
}

for ins in re.split('(\d+)', data[1]):
    if ins == '':
        continue
    elif ins.isnumeric():
        pos = move_forward(matrix, pos, int(ins), dir)
    else:
        dir = dir_lookup[dir][ins]

matrix[pos[0], pos[1]] = 0
#print_board(matrix)
print(1000 * (pos[0] + 1) + 4 * (pos[1] + 1) + dir_score[dir])

146092


In [39]:
corner_lookup = {
    (1, (0,-1)): (4, (0,1)),
    (1, (-1,0)): (6, (0,1)),
    (2, (-1,0)): (6, (-1,0)),
    (2, (0,1)): (5, (0,-1)),
    (2, (1,0)): (3, (0,-1)),
    (3, (0,1)): (2, (-1,0)),
    (3, (0,-1)): (4, (1,0)),
    (4, (-1,0)): (3, (0,1)),
    (4, (0,-1)): (1, (0,1)),
    (5, (0,1)): (2, (0,-1)),
    (5, (1,0)): (6, (0,-1)),
    (6, (0,-1)): (1, (1,0)),
    (6, (0,1)): (5, (-1,0)),
    (6, (1,0)): (2, (1,0)),
}

areas = {
    1: ((0,50), (50,100)),
    2: ((0,50), (100,150)),
    3: ((50,100), (50,100)),
    4: ((100,150), (0,50)),
    5: ((100,150), (50,100)),
    6: ((150,200), (0,50))
}

sides = get_matrix()

sides[0:50, 50:100] = 1
sides[0:50, 100:150] = 2
sides[50:100, 50:100] = 3
sides[100:150, 0:50] = 4
sides[100:150, 50:100] = 5
sides[150:200, 0:50] = 6

corner_funcs = {
    (1, (0,-1)): lambda x: (49 - x[0], x[1]),
    (1, (-1,0)): lambda x: (x[1], x[0]),
    (2, (-1,0)): lambda x: (49 - x[0], x[1]),
    (2, (0,1)): lambda x: (49 - x[0], x[1]),
    (2, (1,0)): lambda x: (x[1], x[0]),
    (3, (0,1)): lambda x: (x[1], x[0]),
    (3, (0,-1)): lambda x: (x[1], x[0]),
    (4, (-1,0)): lambda x: (x[1], x[0]),
    (4, (0,-1)): lambda x: (49 - x[0], x[1]),
    (5, (0,1)): lambda x: (49 - x[0], x[1]),
    (5, (1,0)): lambda x: (x[1], x[0]),
    (6, (0,-1)): lambda x: (x[1], x[0]),
    (6, (0,1)): lambda x: (x[1], x[0]),
    (6, (1,0)): lambda x: (49 - x[0], x[1]),
}

def walk_around_edge(matrix, sides, pos, dir):

    current_side = int(sides[*pos])
    key = (current_side, tuple(dir))
    next_side = corner_lookup[key]

    local_cord = (pos[0] - areas[current_side][0][0], pos[1] - areas[current_side][1][0])
    local_cord_new = corner_funcs[key](local_cord)

    new_cords = (local_cord_new[0] + areas[next_side[0]][0][0], local_cord_new[1] + areas[next_side[0]][1][0])

    return new_cords, next_side[1]

In [43]:
def move_forward(matrix, sides, pos, steps, dir):

    for step in range(steps):
        new_pos = pos + dir

        if (new_pos >= 0).all() and (new_pos < matrix.shape).all():
            point_type = matrix[*new_pos]
        else:
            point_type = ' '

        match point_type:
            case " ":
                new_cords, new_dir = walk_around_edge(matrix, sides, pos, dir)
                point_type = matrix[*new_cords]
                
                if point_type != '#':
                    pos = np.array(new_cords)
                    dir = new_dir
                else:
                    break

            case "#":
                break
            case ".":
                pos = new_pos

    return pos, dir


In [44]:
matrix = get_matrix()

start_col = (matrix[0] != " ").argmax()

pos = np.array([0, start_col])
dir = (0, 1)

for ins in re.split('(\d+)', data[1]):
    if ins == '':
        continue
    elif ins.isnumeric():
        pos, dir = move_forward(matrix, sides, pos, int(ins), dir)
    else:
        dir = dir_lookup[dir][ins]

print(1000 * (pos[0] + 1) + 4 * (pos[1] + 1) + dir_score[dir])

110342
